In [1]:
import pandas as pd
import nltk
import numpy as np
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, TweetTokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from langdetect import detect

from sklearn.naive_bayes import MultinomialNB

from nltk.stem.snowball import SnowballStemmer

from sklearn.ensemble import AdaBoostClassifier #For Classification

from sklearn import metrics

from nltk.collocations import *

from preprocessor import preProcessSerie
import json
import time
import requests
import pickle

#Para el support vector machine:
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold

#Optimización
from sklearn.grid_search import GridSearchCV


#Para kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

#Importar librería del gsi



%matplotlib inline
#%run plot_learning_curve


tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles = True)
tknzrwhu = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles = False)
stemmer=SnowballStemmer("english")


#Creo una variable que contenga todas las palabras de los tweets
#para poder filtrar posteriormente las palabras más frecuentes.
listwords = []

/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#Como la lemmatizer requiere un pos_tag, crearemos este método para
#dárselo y posteriormente haremos lemmatizer en cada tweet
def lemmaSentence(sentence):
    result = ""
    wnl = WordNetLemmatizer()
    poscategory = pos_tag(tknzr.tokenize(sentence))
    for word, tag in poscategory:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a','r','n','v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)
        result = result + " " + lemma
#    print(result)
    return result

#Como la lemmatizer requiere un pos_tag, crearemos este método para
#dárselo y posteriormente lemmatizar las palabras
def lemmaWord(wordsinlist):
    i = 0
    wnl = WordNetLemmatizer()
    poscategory = pos_tag(wordsinlist)
    for word, tag in poscategory:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a','r','n','v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)
        wordsinlist[i] = str(lemma)
        i = i + 1
    return wordsinlist

In [3]:
#Creo un preprocesador que utilizaré posteriormente
#En él trato de eliminar las urls, signos de puntuación y stopwords
#así como palabras como RT, posteriormente hago stemming
def preProcess(serie):
    listatweets = []
    for line in serie:
        frase = ""
        l = ""
        #try:
            #l = bing(line, dst='en')
        #except TranslateError:
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        line = str(emoji_pattern.sub(r'', str(line))) # no emoji
        l = line
        pal = tknzrwhu.tokenize(str(l))
        urls = re.compile(r'.http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        line = urls.sub('', str(line))
        pr = ["rt","@","http","https","'s",'...', 'english', 'translation','):', '. .', '..',
         'à', '؟', 'و', 'ं', 'क', 'ग', 'ज', 'ड', 'त', 'थ', 'द', 'न', 'प', 'ब', 'भ', 'म', '…',
          'य', 'र', 'ल', 'व', 'श', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ै', 'ो',
          '्', 'ാ', 'ി', '്', 'ท', 'พ', 'ั', 'ิ', 'ี', 'ื', 'ู', '็', '่', '้', '์', '–', '—', '‘',
          '’', '“', '”', '、', '。', '「', '」', '【', '】', 'ﷺ', '️', '！', '（', '）', '🏻', '📰', '',
          '📷', '🔥', '🔴', '😂']
        ht = re.compile(r'http.')
        bar = re.compile(r'//*')
        punctuation = set(string.punctuation)
        stoplist = stopwords.words('english')
        #d = enchant.Dict("en_GB")
        pal = [str(i) for i in pal if i not in pr 
            if i not in stoplist if i not in punctuation
            if not bar.search(i) if not ht.search(i)
            if not i.isdigit()]
        #Dentro tenia tb if d.check(str(i))
        [listwords.append(lemmaSentence(i)) for i in pal]
        for p in pal:
            frase = frase + " " + p
        line = lemmaSentence(frase)
        listatweets.append(line)
    return pd.Series(listatweets)

In [4]:
#Creo un preprocesador que utilizaré posteriormente
#En él trato de eliminar las urls, signos de puntuación y stopwords
#así como palabras como RT, posteriormente hago stemming
def preProcessor(tweet):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    tweet = str(emoji_pattern.sub(r'', str(tweet))) # no emoji
    pal = tknzr.tokenize(str(tweet))
    urls = re.compile(r'.http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    line = urls.sub('', str(tweet))
    ht = re.compile(r'http.')
    bar = re.compile(r'//*')
    punctuation = set(string.punctuation)
    stoplist = stopwords.words('english')
    pr = ["rt","@","http","https","'s",'...', 'english', 'translation','):', '. .', '..']
    pal = [stemmer.stem(str(i)) for i in pal if i not in pr 
            if i not in stoplist if i not in punctuation
            if not bar.search(i) if not ht.search(i)
            if not i.isdigit() if not i.startswith('#')]
    #pal = lemmaWord(pal)
    tweet = pal
    return tweet

In [5]:
isis = pd.read_csv('isisfanboy.csv')
about = pd.read_csv('aboutisis.csv')

isis = isis[:17392]
about = about[:17392]
#El máximo que puedo usar es 17392


dataframe = pd.concat([isis, about])
# Define X and Y
X = dataframe['tweets'].values
y = dataframe['radical'].values


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline, FeatureUnion

from collections import Counter 


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer


ngrams_featurizer = Pipeline([
  ('count_vectorizer',  CountVectorizer(ngram_range = (1, 3), encoding = 'ISO-8859-1', 
                                        analyzer=preProcessor)),
  ('tfidf_transformer', TfidfTransformer())
])
    
    
class PosStats(BaseEstimator, TransformerMixin):
    """Obtain number of tokens with POS categories"""

    def stats(self, tweet):
        tokens = tknzr.tokenize(str(tweet))
        tagged = pos_tag(tokens, tagset='universal')
        counts = Counter(tag for word,tag in tagged)
        total = sum(counts.values())
        #copy tags so that we return always the same number of features
        pos_features = {'NOUN': 0, 'ADJ': 0, 'VERB': 0, 'ADV': 0, 'CONJ': 0, 
                        'ADP': 0, 'PRON':0, 'NUM': 0}
        
        pos_dic = dict((tag, float(count)/total) for tag,count in counts.items())
        for k in pos_dic:
            if k in pos_features:
                pos_features[k] = pos_dic[k]
        #print(pos_features)
        return pos_features
    
    def transform(self, data, y=None):
        print("Entra en Pos Stats**")
        dataproc = preProcess(data)
        result = [self.stats(tweet) for tweet in dataproc]
        return result
    
    def fit(self, data, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self

In [8]:
class Hashtags(BaseEstimator, TransformerMixin):
    
    listwords=[]
    
    def fit(self, X, y=None):
        return self
    
    def notinlist(self, item, lista):
        booleano = True
        if item in lista:
            booleano = False
        return booleano    
    
    def allHashtags(self, tweet, lista):
        lista = pickle.load( open( "ht.p", "rb" ) )
        return lista
    
    def Hashtags(self, tweet, listallht, resultlist):
        allhtdict = dict((ht, 0) for ht in listallht)
        sent = tknzrwhu.tokenize(str(tweet))       

        for term in sent:
            if term in listallht:
                allhtdict[term] = 1
        resultlist.append(allhtdict)
        
        return (resultlist)


    def transform(self, data):
        print("Entra en hashtags")
        dataproc = preProcess(data)
        lista = []
        listaresultado = []
        for tweet in dataproc:
            lista = self.allHashtags(tweet, lista)
        for tweet in dataproc:
            listaresultado = self.Hashtags(tweet, lista, listaresultado)
        return listaresultado

In [9]:

from nltk.tag import StanfordNERTagger


st = StanfordNERTagger('/usr/share/stanford-ner-2017-06-09/classifiers/english.all.3class.caseless.distsim.crf.ser.gz', 
                       '/usr/share/stanford-ner-2017-06-09/stanford-ner.jar', 
                      )

st2 = StanfordNERTagger('/usr/share/stanford-ner-2017-06-09/classifiers/english.muc.7class.distsim.crf.ser.gz', 
                       '/usr/share/stanford-ner-2017-06-09/stanford-ner.jar', 
                       )


class NameRecog(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def notinlist(self, item, lista):
        booleano = True
        if item in lista:
            booleano = False
        return booleano
    
    
    def allner(self, dataproc, lista):
        lista = pickle.load( open( "ner.p", "rb" ) )
        return lista
 

    def nameentrec(self, tweet, listallner, listasoloner, resultlist):
        #Inicializo el diccionario siempre, todas las filas tendran los mismos elementos.
        allnerdict = dict((entity, 0) for entity in listallner)
        words = tknzrwhu.tokenize(str(tweet))
        ne_tagged = []
        b = False
        
        for i in words:
            if i.lower() in listasoloner:
                for k in allnerdict:
                    if i.lower() == k[0]:
                        allnerdict[k] = 1
                        
        resultlist.append(allnerdict)
        return (resultlist)

    def transform(self, data):
        dataproc = preProcess(data)
        print("Entra en NER")
        lista = []
        listaresultado = []
        listasolo = []
        lista = self.allner(dataproc,lista)
        for k in lista:
            listasolo.append(k[0])
        for tweet in dataproc:
            listaresultado = self.nameentrec(tweet, lista, listasolo, listaresultado)

        return listaresultado

/home/marioh/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
/home/marioh/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [10]:
from preprocessor import preProcessSerie
import json
import time
import requests
import pickle

class Sentiment(BaseEstimator, TransformerMixin):
    def fit(self, data, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self
    
    def transform(self, data, y=None):
        print("Entra en Sentiments")
        sentdicc = pickle.load( open( "sentdicc.p", "rb" ) )
        lista = []
        for tweet in data:
            tweetsent = sentdicc[tweet]
            lista.append(tweetsent)
        return lista

In [20]:
pipelinesvm = Pipeline([
    ('features', FeatureUnion([
        ('words', TfidfVectorizer(encoding='utf-8', analyzer=preProcessor)),
        ('ngrams', ngrams_featurizer),
        ('pos_stats', Pipeline([
                                ('pos_stats', PosStats()),
                                ('vectors', DictVectorizer())
                            ])),
        ('ner', Pipeline([
            ('nameentrec', NameRecog()),
            ('vectors', DictVectorizer())
        ])),
        ('hashtags', Pipeline([
            ('gethashtags', Hashtags()),
            ('vect', DictVectorizer())
        ])),
        ('sentiments', Pipeline([
            ('getsentiments', Sentiment()),
            ('vector', TfidfVectorizer(encoding='utf-8'))
        ]))
    ])),

    ('clf', SVC(C=10, gamma= 1, kernel='rbf', probability=True)
)

])

#cv = KFold(X.shape[0], 4, shuffle=True, random_state=33)
#scores = cross_val_score(pipeline, X, y, cv=cv)
#print("Scores in every iteration", scores)
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

pipelinesvm.fit(X,y)

/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Entra en Pos Stats**
Entra en NER
Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#armed': 0, '#العریش': 0, '#micropoetry': 0, '#kabarsumsel': 0, '#radicalised': 0, '#kurigram': 0, '#海外の反応': 0, '#schmuck': 0, '#terrorist': 0, '#تفجير_المدينة': 0, '#mansoura': 0, '#spectacles': 0, '#whatsapp': 0, '#mumbai': 0, '#tribes': 0, '#deirezzawr': 0, '#fbi': 0, '#sunnies': 0, '#migration': 0, '#terroristassad': 0, '#migrantcrisis': 0, '#ابيـــة': 0, '#uqayribat': 0, '#sdf': 0, '#missiles': 0, '#khanaseer': 0, '#الدولة_الأســـلامية': 0, '#kenyan': 0, '#cyprus': 0, '#scandal': 0, '#akantony': 0, '#السفيرة': 0, '#alshababba': 0, '#nukhayl': 0, '#first': 0, '#terrorarrest': 0, '#saham_jolan': 0, '#president': 0, '#dam': 0, '#mayadin': 0, '#malayalamnews': 0, '#jewsofjihad': 0, '#abrini': 0, '#pakvsl': 0, '#bucciovertimechallenge': 0, '#mujahid': 0, '#youtube': 0, '#yaromuk': 0, '#social2': 0, "#baqba's": 0, '#人質事件': 0, '#lattakia': 0, '#fuahkafray

/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('words', TfidfVectorizer(analyzer=<function preProcessor at 0x7f3b92531ae8>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        m...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [11]:
isisnum = pd.read_csv('isisfanboynum.csv')
aboutnum = pd.read_csv('aboutisisnum.csv')

isisnum = isisnum[:17392]
aboutnum = aboutnum[:17392]
#El máximo que puedo usar es 17392


dataframenum = pd.concat([isisnum, aboutnum])
# Define X and Y
Xnum = dataframenum['tweets'].values
ynum = dataframenum['radical'].values

print(np.unique(ynum))
pipelinesvmwordsnum = Pipeline([
    ('words', TfidfVectorizer(encoding='utf-8', analyzer=preProcessor)),
    ('clf', SVC(C=10, gamma= 1, kernel='rbf', probability=True))

])
print("Hola, ha entrado")
cv = KFold(X.shape[0], 10, shuffle=True, random_state=33)
scoressvmwordsnum = cross_val_score(pipelinesvmwordsnum, Xnum, ynum, cv=cv)
print("Scores in every iteration", scoressvmwordsnum)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoressvmwordsnum.mean(), scoressvmwordsnum.std() * 2))

[0 1]
Hola, ha entrado


/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/marioh/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float)

Scores in every iteration [ 0.94739868  0.95688416  0.9534349   0.96032202  0.94968373  0.95917194
  0.95543416  0.95830937  0.95658424  0.95342151]
Accuracy: 0.96 (+/- 0.01)


In [ ]:
pipelinekpos = Pipeline([
    ('pos_stats', Pipeline([
        ('pos_stats', PosStats()),
        ('vectors', DictVectorizer())
    ])),
    ('modelknn', KNeighborsClassifier(n_neighbors = 13))

])

cv = KFold(X.shape[0], 10, shuffle=True, random_state=33)
scoreskpos = cross_val_score(pipelinekpos, X, y, cv=cv)
print("Scores in every iteration", scoreskpos)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoreskpos.mean(), scoreskpos.std() * 2))

In [18]:
pipelinenbht = Pipeline([
    ('hashtags', Pipeline([
        ('gethashtags', Hashtags()),
        ('vect', DictVectorizer())
    ])),
    ('clf', AdaBoostClassifier(n_estimators=195, base_estimator=MultinomialNB(alpha=.01), learning_rate=1))

])

cv = KFold(X.shape[0], 10, shuffle=True, random_state=33)
scoresnbht = cross_val_score(pipelinenbht, X, y, cv=cv)
print("Scores in every iteration", scoresnbht)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresnbht.mean(), scoresnbht.std() * 2))

Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis': 0, '#صلاح_الدين':

Entra en hashtags
ESTE ---------------------------------------------------
#pokemongo
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis': 0, '#صلاح_الدين': 0, '#bayji':

Entra en hashtags
Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis':

Entra en hashtags
Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis':

Entra en hashtags
ESTE ---------------------------------------------------
#mustread
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis': 0, '#صلاح_الدين': 0, '#bayji': 

...

Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis': 0, '#صلاح_الدين':

Entra en hashtags
Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis':

Entra en hashtags
Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis':

Entra en hashtags
Entra en hashtags
ESTE ---------------------------------------------------
#indiaisisandbangladesh
{'#dontbombsyria': 0, '#narrativa': 0, '#darkweb': 0, '#glimpse': 0, '#banias_butcher': 0, '#conservitivebecause': 0, '#tannouza': 0, '#us_airstrikes': 0, '#abusayyaf': 0, '#madaya_is_starving': 0, '#indian': 0, '#doyourpart': 0, '#prayformecca': 0, '#دير': 0, '#personnel': 0, '#booksforsisters': 0, '#ثورة_سوريا_المباركة': 0, '#huge': 0, '#daraya': 0, '#youtube': 0, '#saudi-linked': 0, '#makhoul': 0, '#bulgaria': 0, '#murtaddin': 0, '#jackletts': 0, '#alhayatmediacenter': 0, '#future': 0, '#nursecrisis': 0, '#la': 0, '#vivanews': 0, '#proudmuslim': 0, '#occupywallstreet': 0, '#initial15': 0, '#starwars': 0, '#teleiis': 0, '#ypj': 0, '#japan': 0, '#viaalterna': 0, '#dozens': 0, '#ra': 0, '#pyd': 0, '#adnani': 0, '#وين_اعدام_الارهابيين': 0, '#turchia': 0, '#indonesia': 0, '#ttp': 0, '#negarse': 0, '#terrorismo': 0, '#freemusacerantonio': 0, '#migrantcrisis': 0, '#iceisis':

Entra en hashtags
Scores in every iteration [ 0.  1.  0.  0.  1.  0.  1.  1.  0.  1.]
Accuracy: 0.50 (+/- 1.00)


In [ ]:
pipelinesvmht = Pipeline([
    ('hashtags', Pipeline([
        ('gethashtags', Hashtags()),
        ('vect', DictVectorizer())
    ])),
    ('clf', SVC(C=10, gamma= 1, kernel='rbf', probability=True))

])

cv = KFold(X.shape[0], 10, shuffle=True, random_state=33)
scoressvmht = cross_val_score(pipelinesvmht, X, y, cv=cv)
print("Scores in every iteration", scoressvmht)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoressvmht.mean(), scoressvmht.std() * 2))

In [ ]:
pipelinenbsent = Pipeline([
    ('sentiments', Pipeline([
        ('getsentiments', Sentiment()),
        ('vector', TfidfVectorizer(encoding='utf-8'))
        ]))
    ('clf', AdaBoostClassifier(n_estimators=195, base_estimator=MultinomialNB(alpha=.01), learning_rate=1))

])

cv = KFold(X.shape[0], 10, shuffle=True, random_state=33)
scoresnbsent = cross_val_score(pipelinenbsent, X, y, cv=cv)
print("Scores in every iteration", scoresnbsent)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresnbsent.mean(), scoresnbsent.std() * 2))

In [ ]:
pipelinesvmsent = Pipeline([
    ('sentiments', Pipeline([
        ('getsentiments', Sentiment()),
        ('vector', TfidfVectorizer(encoding='utf-8'))
    ]))
    ('clf', SVC(C=10, gamma= 1, kernel='rbf', probability=True))

])

cv = KFold(X.shape[0], 10, shuffle=True, random_state=33)
scoressvmsent = cross_val_score(pipelinesvmsent, X, y, cv=cv)
print("Scores in every iteration", scoressvmsent)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoressvmsent.mean(), scoressvmsent.std() * 2))